In [8]:
# Extracting the S&P 500 stocks 
import yfinance as yf 
import pandas as pd
from itertools import combinations
from skopt import gp_minimize
import random
from scipy.stats import pearsonr
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from matplotlib import pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

SEED = 8
random.seed(SEED)
np.random.seed(SEED)


# Run it if u get an error saying u do not have "skopt"
# !pip install scikit-optimize

In [9]:
def fetch_sp500_data(start_date="2023-01-01", end_date="2024-01-01"):
    """
    Fetches S&P 500 stocks from Wikipedia, then downloads daily price data using yfinance.
    Returns the dataframe containing the stock data.
    """
    import yfinance as yf
    import pandas as pd
    
    # Step 1: Get S&P 500 tickers
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    sp500_table = pd.read_html(url)[0]
    sp500_tickers = sp500_table["Symbol"].tolist()
    
    # Step 2: Fetch stock data
    sp500_data = yf.download(sp500_tickers, interval="1d", start=start_date, end=end_date, auto_adjust=False)
    
    print(sp500_data.head())
    return sp500_data

In [10]:
def prepare_close_prices_and_pairs(sp500_data, max_na_threshold=1):
    """
    Extracts the Close prices from the S&P 500 data, removes columns with excessive NaN values,
    and generates all possible stock pairs.

    Parameters:
        sp500_data (DataFrame): The full S&P 500 stock data.
        max_na_threshold (int): Maximum allowed NaN values per column. Columns with more NaNs are dropped.

    Returns:
        close_price (DataFrame): Cleaned DataFrame containing only the Close prices.
        stock_pairs (list): List of all possible stock pairs (combinations of tickers).
        tickers (list): List of tickers after cleaning.
    """
    # Extract Close prices
    close_price = sp500_data["Close"]
    
    # Identify columns with excessive NaN values
    list_of_na_columns = [col for col in close_price if close_price[col].isna().sum() > max_na_threshold]
    
    # Drop columns with excessive NaN values
    close_price = close_price.drop(columns=list_of_na_columns)
    
    # Generate all possible stock pairs
    stock_pairs = list(combinations(close_price.columns, 2))
    
    # Get the list of tickers
    tickers = close_price.columns.tolist()
    
    return close_price, stock_pairs, tickers

# close_price, stock_pairs, tickers = prepare_close_prices_and_pairs(sp500_data)

In [11]:
def calculate_indicators(close_price):
    """
    Calculates SMA-5, SMA-10, and 10-day rolling volatility for all stocks in the given Close price DataFrame.

    Parameters:
        close_price (DataFrame): DataFrame containing Close prices of stocks.

    Returns:
        combined_df (DataFrame): A MultiIndex DataFrame containing Close prices, SMA-5, SMA-10, and Volatility.
    """
    # Base DataFrame with MultiIndex for Close prices
    price_df = close_price.copy()
    price_df.columns = pd.MultiIndex.from_product([['Close'], price_df.columns])

    # Containers for multiple SMAs and volatility
    sma_5_cols = []
    sma_10_cols = []
    volatility_cols = []

    # Calculate daily returns
    returns = close_price.pct_change()

    # Calculate 10-day rolling volatility
    for ticker in close_price.columns:
        vol = returns[ticker].rolling(window=10).std()
        volatility_cols.append(vol.rename(ticker))

    # Combine volatility into MultiIndex DataFrame
    volatility_df = pd.concat(volatility_cols, axis=1)
    volatility_df.columns = pd.MultiIndex.from_product([['Volatility'], volatility_df.columns])

    # Calculate SMA-5 and SMA-10 for all stocks
    for ticker in close_price.columns:
        series = close_price[ticker]
        
        # SMA-5
        sma_5 = series.rolling(window=5).mean()
        sma_5_cols.append(sma_5.rename(ticker))
        
        # SMA-10
        sma_10 = series.rolling(window=10).mean()
        sma_10_cols.append(sma_10.rename(ticker))

    # Combine SMA-5 into MultiIndex DataFrame
    sma_5_df = pd.concat(sma_5_cols, axis=1)
    sma_5_df.columns = pd.MultiIndex.from_product([['SMA-5'], sma_5_df.columns])

    # Combine SMA-10 into MultiIndex DataFrame
    sma_10_df = pd.concat(sma_10_cols, axis=1)
    sma_10_df.columns = pd.MultiIndex.from_product([['SMA-10'], sma_10_df.columns])

    # Merge all indicators into a single DataFrame
    combined_df = pd.concat([price_df, sma_5_df, sma_10_df, volatility_df], axis=1)

    return combined_df

# combined_df = calculate_indicators(close_price)

In [12]:
def find_cointegrated_pairs(combined_df, tickers, cointegration_threshold=0.05, n_calls=50, random_state=42):
    """
    Finds cointegrated pairs of stocks using Bayesian Optimization.

    Parameters:
        combined_df (DataFrame): MultiIndex DataFrame containing Close prices, SMA-5, SMA-10, and Volatility.
        tickers (list): List of stock tickers.
        cointegration_threshold (float): Threshold for cointegration p-value to consider a pair significant.
        n_calls (int): Number of calls for Bayesian Optimization.
        random_state (int): Random state for reproducibility.

    Returns:
        found_pairs (list): List of tuples containing cointegrated pairs and their p-values.
    """
    remaining_tickers = tickers.copy()
    found_pairs = []
    evaluated_pairs = set()

    def cointegration_test(params):
        idx1, idx2 = sorted([int(params[0]), int(params[1])])
        pair = (idx1, idx2)

        if idx1 == idx2 or pair in evaluated_pairs:
            return 1.0

        evaluated_pairs.add(pair)
        stock1, stock2 = remaining_tickers[idx1], remaining_tickers[idx2]

        s1_close = combined_df[('Close', stock1)].dropna()
        s2_close = combined_df[('Close', stock2)].dropna()
        s1_sma5 = combined_df[('SMA-5', stock1)].dropna()
        s2_sma5 = combined_df[('SMA-5', stock2)].dropna()
        s1_sma10 = combined_df[('SMA-10', stock1)].dropna()
        s2_sma10 = combined_df[('SMA-10', stock2)].dropna()
        s1_vol = combined_df[('Volatility', stock1)].dropna()
        s2_vol = combined_df[('Volatility', stock2)].dropna()

        aligned = pd.concat([s1_close, s2_close, s1_sma5, s2_sma5, s1_sma10, s2_sma10,
                             s1_vol, s2_vol], axis=1, join='inner').dropna()

        if len(aligned) < 30:
            return 1.0

        s1_close = aligned.iloc[:, 0]
        s2_close = aligned.iloc[:, 1]
        s1_sma5 = aligned.iloc[:, 2]
        s2_sma5 = aligned.iloc[:, 3]
        s1_sma10 = aligned.iloc[:, 4]
        s2_sma10 = aligned.iloc[:, 5]
        s1_vol = aligned.iloc[:, 6]
        s2_vol = aligned.iloc[:, 7]

        # Calculate SMA correlation
        sma5_corr = s1_sma5.corr(s2_sma5)
        sma10_corr = s1_sma10.corr(s2_sma10)

        if np.isnan(sma5_corr):
            sma5_corr = 0
        if np.isnan(sma10_corr):
            sma10_corr = 0

        # Normalize vol using mean volatility
        mean_vol = (s1_vol.mean() + s2_vol.mean()) / 2
        if mean_vol == 0:
            vol_penalty = 1.0  # Arbitrary strong penalty
        else:
            vol_penalty = abs(s1_vol.mean() - s2_vol.mean()) / mean_vol

        avg_sma_corr = (sma5_corr + sma10_corr) / 2  # Simple average of the two correlations

        # Cointegration p-value
        _, pvalue, _ = coint(s1_close, s2_close)

        # Combine into one score for BO to minimize
        alpha = 0.2  # weight for SMA correlation
        beta = 0.1   # weight for volatility mismatch

        score = pvalue + alpha * (1 - avg_sma_corr) + beta * vol_penalty

        return score

    # Greedy search loop
    while len(remaining_tickers) > 2:
        num_stocks = len(remaining_tickers)

        if num_stocks < 2:
            break

        # Define BO search space
        search_space = [(0, num_stocks - 1), (0, num_stocks - 1)]

        # Run Bayesian Optimization
        result = gp_minimize(cointegration_test, search_space, n_calls=n_calls, random_state=random_state)
        best_idx = sorted(result.x)
        best_pair = (remaining_tickers[best_idx[0]], remaining_tickers[best_idx[1]])
        best_pvalue = result.fun

        # Store only significant cointegrated pairs
        if best_pvalue < cointegration_threshold:
            print(f"✅ Cointegrated Pair Found: {best_pair} | p-value = {best_pvalue:.4f}")
            found_pairs.append((best_pair, best_pvalue))

        # Remove used tickers to avoid duplication
        remaining_tickers.remove(best_pair[0])
        remaining_tickers.remove(best_pair[1])

    # Final summary
    print(f"\n🎯 Total Cointegrated Pairs Found: {len(found_pairs)}")
    for pair, pval in found_pairs:
        print(f"{pair} | p = {pval:.4f}")

    return found_pairs

# co_pairs = find_cointegrated_pairs(combined_df, tickers)

In [13]:
yearly_cointegrations = {}

# Loop through 2019 start - 2020 start TO 2023 start - 2024 start
for year in range(2019, 2024):
    start_date = f"{year}-01-01"
    end_date = f"{year+1}-01-01"
    print(f"==== Processing data for {start_date} to {end_date} ====")
    
    # Use the extraction function
    sp500_data_year = fetch_sp500_data(start_date, end_date)
    
    # Prepare data
    close_price_year, stock_pairs_year, tickers_year = prepare_close_prices_and_pairs(sp500_data_year)
    combined_df_year = calculate_indicators(close_price_year)
    
    # Find cointegrated pairs for this year
    co_pairs_year = find_cointegrated_pairs(combined_df_year, tickers_year)
    yearly_cointegrations[year] = co_pairs_year

print("All yearly cointegration results:", yearly_cointegrations)

==== Processing data for 2019-01-01 to 2020-01-01 ====


[*********************100%***********************]  503 of 503 completed

16 Failed downloads:
['VLTO', 'EXE', 'OTIS', 'CEG', 'CARR', 'SOLV', 'SW', 'GEHC', 'KVUE', 'PLTR', 'GEV', 'ABNB', 'DASH']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2020-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1546318800, endDate = 1577854800")')
['BF.B', 'WDC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2020-01-01)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Price       Adj Close                                                   \
Ticker              A       AAPL       ABBV ABNB        ABT       ACGL   
Date                                                                     
2019-01-02  62.944958  37.667202  67.518402  NaN  62.330669  24.904034   
2019-01-03  60.626076  33.915257  65.293770  NaN  59.389023  24.514164   
2019-01-04  62.724556  35.363071  67.397339  NaN  61.084068  25.094212   
2019-01-07  64.056473  35.284363  68.381027  NaN  61.998833  25.037159   
2019-01-08  64.995537  35.956993  68.698807  NaN  61.254475  25.132248   

Price                                                     ...  Volume  \
Ticker             ACN        ADBE        ADI        ADM  ...     WTW   
Date                                                      ...           
2019-01-02  129.003677  224.570007  76.502014  34.205593  ...  743800   
2019-01-03  124.599236  215.699997  71.880898  34.055241  ...  689800   
2019-01-04  129.444122  226.190002  73.626

[*********************100%***********************]  503 of 503 completed

11 Failed downloads:
['VLTO', 'EXE', 'CEG', 'SOLV', 'SW', 'GEHC', 'KVUE', 'GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2021-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1577854800, endDate = 1609477200")')
['BF.B', 'WDC']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2021-01-01)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Price       Adj Close                                                   \
Ticker              A       AAPL       ABBV ABNB        ABT       ACGL   
Date                                                                     
2020-01-02  83.061333  72.716087  71.589783  NaN  79.273956  41.268997   
2020-01-03  81.727730  72.009125  70.910248  NaN  78.307510  41.221451   
2020-01-06  81.969322  72.582916  71.469856  NaN  78.717804  41.383106   
2020-01-07  82.220581  72.241554  71.062164  NaN  78.280167  41.040779   
2020-01-08  83.032349  73.403648  71.565796  NaN  78.599274  40.631893   

Price                                                      ...  Volume  \
Ticker             ACN        ADBE         ADI        ADM  ...     WTW   
Date                                                       ...           
2020-01-02  195.263565  334.429993  109.436218  39.867687  ...  465800   
2020-01-03  194.938370  331.809998  107.509750  39.789875  ...  411400   
2020-01-06  193.665405  333.709991  1

...

Price        Adj Close                                                 \
Ticker               A        AAPL       ABBV        ABNB         ABT   
Date                                                                    
2021-01-04  115.582764  126.405251  88.933960  139.149994  101.045967   
2021-01-05  116.527771  127.968071  89.853584  148.300003  102.296181   
2021-01-06  119.723228  123.660484  89.077385  142.770004  102.083191   
2021-01-07  122.908981  127.880165  90.030762  151.270004  103.074112   
2021-01-08  123.785789  128.983948  90.503220  149.770004  103.361191   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2021-01-04  33.186359  242.118515  485.339996  135.426224  44.578201  ...   
2021-01-05  33.319485  243.496872  485.690002  137.846832  45.445274  ...   
2021-01-06  34.783871  246.159

[*********************100%***********************]  503 of 503 completed

8 Failed downloads:
['VLTO', 'SOLV', 'SW', 'KVUE', 'GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2023-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1641013200, endDate = 1672549200")')
['BF.B', 'LRCX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-01-01 -> 2023-01-01)')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Price        Adj Close                                                  \
Ticker               A        AAPL        ABBV        ABNB         ABT   
Date                                                                     
2022-01-03  153.272278  178.879898  119.733437  172.679993  130.754242   
2022-01-04  148.090729  176.609650  119.503555  170.800003  127.679146   
2022-01-05  145.553818  171.911835  120.131332  162.250000  127.105469   
2022-01-06  146.063141  169.042084  119.565460  159.750000  127.086632   
2022-01-07  142.174515  169.209137  119.256020  166.050003  127.481651   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2022-01-03  42.362530  389.190216  564.369995  167.036636  62.154781  ...   
2022-01-04  42.914051  386.409058  554.000000  165.527969  63.308331  ...   
2022-01-05  42.410072 

[*********************100%***********************]  503 of 503 completed

6 Failed downloads:
['BF.B', 'LRCX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-01-01 -> 2024-01-01)')
['SOLV', 'SW', 'GEV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1672549200, endDate = 1704085200")')
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Price        Adj Close                                                 \
Ticker               A        AAPL        ABBV       ABNB         ABT   
Date                                                                    
2023-01-03  147.931244  123.632530  149.164093  84.900002  104.779045   
2023-01-04  149.538361  124.907707  150.367493  88.720001  106.337631   
2023-01-05  149.972153  123.583092  150.183777  87.709999  105.945610   
2023-01-06  145.594528  128.130249  152.994736  88.519997  107.408569   
2023-01-09  145.397354  128.654160  148.502716  89.239998  107.236450   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2023-01-03  59.393120  261.824371  336.920013  156.076004  83.634407  ...   
2023-01-04  59.687901  260.933044  341.410004  159.400269  80.593147  ...   
2023-01-05  59.849552  254.771

In [14]:
import json

# Export yearly_cointegrations as a JSON file
with open("yearly_cointegrations.json", "w") as json_file:
    json.dump(yearly_cointegrations, json_file, indent=4)

print("All yearly cointegration results have been saved to 'yearly_cointegrations.json'.")

All yearly cointegration results have been saved to 'yearly_cointegrations.json'.
